## Imports

In [ ]:
# pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
from platform import python_version

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config


set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24
print("Python", python_version())

Pandas   1.2.4
Sklearn  0.24.1
Python 3.8.8


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


In [4]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [7]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [8]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [9]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [10]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df['Title'] = df['Title'].map(title_dictionary)
df_test['Title'] = df_test['Title'].map(title_dictionary)

In [11]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [13]:
df['Cabin'] = df['Cabin'].notna().astype(int)
df_test['Cabin'] = df_test['Cabin'].notna().astype(int)

In [14]:
df, df_test

(             Survived  Pclass  \
 PassengerId                     
 1                   0       3   
 2                   1       1   
 3                   1       3   
 4                   1       1   
 5                   0       3   
 ...               ...     ...   
 887                 0       2   
 888                 1       1   
 889                 0       3   
 890                 1       1   
 891                 0       3   
 
                                                           Name     Sex   Age  \
 PassengerId                                                                    
 1                                      Braund, Mr. Owen Harris    male  22.0   
 2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
 3                                       Heikkinen, Miss. Laina  female  26.0   
 4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
 5                                     Allen, Mr. William Henry    mal

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [15]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [16]:
x.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin         0
Embarked      2
Title         0
dtype: int64

In [17]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'Cabin'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'Cabin']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [55]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE
from sklearn.experimental import enable_iterative_imputer

num_4_treeModels = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=True)), # mean, median
])

cat_4_treeModels = pipeline.Pipeline(steps=[  
  # Some SimpleImputer here
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing', add_indicator=True)),
  # Some Encoder here. Remember to handle_unknown
  ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=-9999))
  # ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='ignore'))
])

num_4_multModels = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), # mean, median
  ('imputer', impute.KNNImputer(n_neighbors=10, add_indicator=True)), # mean, median
  ('quant', preprocessing.QuantileTransformer(output_distribution='normal', random_state=73)),
  ('norm', preprocessing.Normalizer()),
])

cat_4_multModels = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing', add_indicator=True)),
  # Some Encoder here. Remember to handle_unknown
  ('onehot', preprocessing.OneHotEncoder(categories='auto', handle_unknown='ignore')),
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num_t', num_4_treeModels, num_vars),
    ('cat_t', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

mult_prepro = compose.ColumnTransformer(transformers=[
    ('num_m', num_4_multModels, num_vars),
    ('cat_m', cat_4_multModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars



In [56]:
mult_prepro

ColumnTransformer(transformers=[('num_m',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(add_indicator=True,
                                                             n_neighbors=10)),
                                                 ('quant',
                                                  QuantileTransformer(output_distribution='normal',
                                                                      random_state=73))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age',
                                  'Cabin']),
                                ('cat_m',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(add_indicator=True,
                                                                fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [57]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [58]:
from sklearn.tree           import DecisionTreeClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import ExtraTreesClassifier
from sklearn.ensemble       import AdaBoostClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.experimental   import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble       import HistGradientBoostingClassifier
from xgboost                import XGBClassifier
from lightgbm               import LGBMClassifier
from catboost               import CatBoostClassifier

from sklearn.svm            import SVC
from sklearn.linear_model   import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.naive_bayes    import CategoricalNB




In [59]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":
  "Random Forest":
  "AdaBoost":
  "Skl GBM":
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
""";


# YOUR CODE HERE

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
  "Extra Trees": ExtraTreesClassifier(n_estimators=200),
  "Random Forest": RandomForestClassifier(n_estimators=200),
  "AdaBoost": AdaBoostClassifier(n_estimators=200),
  "Skl GBM": GradientBoostingClassifier(n_estimators=200),
  "Skl HistGBM": HistGradientBoostingClassifier(random_state=0),
  "XGBoost": XGBClassifier(random_state=0),
  "LightGBM": LGBMClassifier(random_state=0),
  "CatBoost":CatBoostClassifier(random_state=0),
  }

mult_classifiers = {
  "SVC": SVC(random_state=0),
  "Logistic": LogisticRegression(random_state=0),
  "Neural": MLPClassifier(),
  "KNN": KNeighborsClassifier(n_neighbors=10),
  "Naive Bayes": CategoricalNB()
}


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

mult_classifiers = {name: pipeline.make_pipeline(mult_prepro, model) for name, model in mult_classifiers.items()}

mult_classifiers["Neural"]



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_m',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True,
                                                                              n_neighbors=10)),
                                                                  ('quant',
                                                                   QuantileTransformer(output_distribution='normal',
                                                                                       random_state=73))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age', 'Cabin']),
                                                 ('cat_m',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('mlpclassifier', MLPClassifier())])

In [60]:
classifiers = {**tree_classifiers, **mult_classifiers}

classifiers['Neural']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_m',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True,
                                                                              n_neighbors=10)),
                                                                  ('quant',
                                                                   QuantileTransformer(output_distribution='normal',
                                                                                       random_state=73))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age', 'Cabin']),
                                                 ('cat_m',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('mlpclassifier', MLPClassifier())])

In [61]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [62]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    train_size=0.8,
    stratify=y,
    random_state=0
)

# x_train = x_train.values
# x_val = x_val.values
# y_train = y_train.values
# y_val = y_val.values

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in classifiers.items():
    start_time = time.time()

    model.fit(x_train, y_train)

    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)




results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

61:	learn: 0.2936297	total: 662ms	remaining: 338ms
662:	learn: 0.2935220	total: 663ms	remaining: 337ms
663:	learn: 0.2934123	total: 664ms	remaining: 336ms
664:	learn: 0.2933400	total: 665ms	remaining: 335ms
665:	learn: 0.2932735	total: 666ms	remaining: 334ms
666:	learn: 0.2930786	total: 667ms	remaining: 333ms
667:	learn: 0.2930100	total: 668ms	remaining: 332ms
668:	learn: 0.2928777	total: 669ms	remaining: 331ms
669:	learn: 0.2928637	total: 669ms	remaining: 330ms
670:	learn: 0.2928177	total: 670ms	remaining: 329ms
671:	learn: 0.2928019	total: 671ms	remaining: 328ms
672:	learn: 0.2926375	total: 672ms	remaining: 327ms
673:	learn: 0.2925081	total: 673ms	remaining: 326ms
674:	learn: 0.2923896	total: 674ms	remaining: 324ms
675:	learn: 0.2922942	total: 675ms	remaining: 323ms
676:	learn: 0.2922625	total: 676ms	remaining: 322ms
677:	learn: 0.2921641	total: 677ms	remaining: 321ms
678:	learn: 0.2920876	total: 678ms	remaining: 320ms
679:	learn: 0.2920191	total: 679ms	remaining: 319ms
680:	learn: 0

ValueError: Negative values in data passed to CategoricalNB (input X)

In [ ]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) >= 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [28]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
# """
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=0
)
skf.split(x, y)
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
""";

# for train_index, val_index in skf.split(x_fold, y_fold):
#     # print("TRAIN:", train_index, "TEST:", val_index)
#     x_train, x_val = x_fold[train_index], x_fold[val_index]
#     y_train, y_val = y_fold[train_index], y_fold[val_index]



# YOUR CODE HERE
for model_name, model in classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y

    pred = model_selection.cross_val_predict(model, x, y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                            "Accuracy": metrics.accuracy_score(y, pred)*100,
                            "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                            "Time":     total_time},
                            ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

tal: 633ms	remaining: 68.8ms
902:	learn: 0.2777468	total: 634ms	remaining: 68.1ms
903:	learn: 0.2776871	total: 634ms	remaining: 67.4ms
904:	learn: 0.2776584	total: 635ms	remaining: 66.7ms
905:	learn: 0.2775982	total: 636ms	remaining: 66ms
906:	learn: 0.2775220	total: 637ms	remaining: 65.3ms
907:	learn: 0.2774252	total: 637ms	remaining: 64.6ms
908:	learn: 0.2773652	total: 638ms	remaining: 63.9ms
909:	learn: 0.2772933	total: 639ms	remaining: 63.2ms
910:	learn: 0.2772232	total: 640ms	remaining: 62.5ms
911:	learn: 0.2771706	total: 640ms	remaining: 61.8ms
912:	learn: 0.2771071	total: 641ms	remaining: 61.1ms
913:	learn: 0.2769870	total: 642ms	remaining: 60.4ms
914:	learn: 0.2769105	total: 642ms	remaining: 59.7ms
915:	learn: 0.2767064	total: 643ms	remaining: 59ms
916:	learn: 0.2766743	total: 648ms	remaining: 58.6ms
917:	learn: 0.2765823	total: 649ms	remaining: 58ms
918:	learn: 0.2765224	total: 650ms	remaining: 57.3ms
919:	learn: 0.2764303	total: 650ms	remaining: 56.6ms
920:	learn: 0.2763395	t

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,84.175084,82.142172,1.677716
2,LightGBM,82.940516,81.250599,0.631677
3,CatBoost,82.603816,80.701754,8.074305
4,Skl HistGBM,82.042649,80.191257,3.068537
5,SVC,82.042649,79.364395,0.534172
6,KNN,82.042649,79.088774,0.429464
7,Logistic,81.818182,80.284728,0.438815
8,XGBoost,81.593715,79.992331,0.892453
9,Neural,81.481481,79.294890,3.839267
10,Random Forest,81.369248,79.920429,2.880591


In [30]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) >= 9

# Exercise 7.1
Train with all data the best model

In [32]:
best_model = classifiers["SVC"]

# Fit best model with all data

# YOUR CODE HERE
best_model.fit(x, y)

/home/alessio/anaconda3/envs/machine_learning/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_m',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True,
                                                                              n_neighbors=10)),
                                                                  ('quant',
                                                                   QuantileTransformer(output_distribution='normal',
                                                                                       random_state=73)),
                                                                  ('norm',
                                                                   Normalizer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age', 'Cabin']),
                                                 ('cat_m',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('svc', SVC(random_state=0))])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [33]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [34]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [35]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,0


In [36]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"